# Traffic Monitoring System

This notebook allows you to run the Traffic Monitoring System on platforms like Google Colab, Kaggle, or any Jupyter environment. The system can:

- **Detect Vehicles in Real-Time**: Spot cars, trucks, and other vehicles in video feeds
- **Read License Plates**: Automatically identify and record license plate numbers
- **Track Moving Vehicles**: Follow each vehicle as it moves through the video
- **Count Traffic**: Count vehicles as they cross a line you define on the screen
- **Store Results**: Save all detection data for later analysis

## How It Works

1. **Video Ingestion**: Gets frames from your video source
2. **Detection**: Uses AI to find vehicles and license plates
3. **Tracking**: Keeps track of each vehicle as it moves
4. **Counting**: Counts vehicles when they cross your defined line
5. **OCR**: Reads license plate text when detected
6. **Storage**: Saves results to a database
7. **Main Application**: Coordinates all the components and shows results

## 1. Setup Environment

First, let's set up the environment and install all necessary dependencies.

In [ ]:
# Check if we're running in Colab
import sys
IN_COLAB = 'google.colab' in sys.modules
IN_KAGGLE = 'kaggle' in sys.modules

print(f"Running in Google Colab: {IN_COLAB}")
print(f"Running in Kaggle: {IN_KAGGLE}")

In [ ]:
# Install necessary packages
!pip install opencv-python-headless numpy ultralytics onnxruntime boxmot easyocr paho-mqtt SQLite3 tqdm

In [ ]:
# Clone the repository if we're in Colab or Kaggle
if IN_COLAB or IN_KAGGLE or not any('traffic_monitoring' in s for s in !ls):
    !git clone https://github.com/yourusername/traffic_monitoring.git
    %cd traffic_monitoring
else:
    print("Repository already exists or we're running locally")

## 2. Download and Prepare Models

Now, let's download and prepare the AI models needed for vehicle detection and license plate recognition.

In [ ]:
import os
from pathlib import Path

# Create models directory if it doesn't exist
os.makedirs('models', exist_ok=True)

# Check if models already exist
vehicle_model_path = Path('./models/yolo11s.onnx')
plate_model_path = Path('./models/plate_v8n.onnx')

if not vehicle_model_path.exists() or not plate_model_path.exists():
    print("Downloading models...")
    # Download vehicle detection model
    !python -c "from ultralytics import YOLO; YOLO('yolo11s.pt')"
    # Move the downloaded model to our models directory
    !mv yolo11s.pt models/ 2>/dev/null || echo "Model may already be in place"
    
    # Download plate detection model (this is a simplified example - adjust as needed)
    !wget -q https://github.com/yourusername/traffic_monitoring/releases/download/v1.0/plate_v8n.pt -O models/plate_v8n.pt
    
    # Convert models to ONNX format for better performance
    !python utils/model_converter.py --model models/yolo11s.pt --output models/yolo11s.onnx
    !python utils/model_converter.py --model models/plate_v8n.pt --output models/plate_v8n.onnx
else:
    print("Models already exist, skipping download")

## 3. Upload a Video or Use Sample Data

Now you can either upload your own video or use one of the sample videos.

In [ ]:
# Create a data directory if it doesn't exist
os.makedirs('data', exist_ok=True)

# For Colab: provide an upload widget
if IN_COLAB:
    from google.colab import files
    
    print("Please upload a video file:")
    uploaded = files.upload()
    
    for filename in uploaded.keys():
        video_path = os.path.join('data', filename)
        with open(video_path, 'wb') as f:
            f.write(uploaded[filename])
        print(f"Uploaded {filename} to {video_path}")
elif IN_KAGGLE:
    # For Kaggle: list available input files
    print("Available input files:")
    !ls ../input
    
    # Symlink input directory to make files accessible
    !ln -s ../input input
else:
    print("Available sample videos:")
    !ls data/*.mp4 data/*.MOV data/*.avi 2>/dev/null || echo "No sample videos found in data/ directory"

## 4. Configure the System

Let's set up the configuration for the Traffic Monitoring System.

In [ ]:
# Create a simplified config for Jupyter environment
import yaml
import os

# Make sure config directory exists
os.makedirs('config/settings', exist_ok=True)

# Create a basic configuration
config = {
    'video': {
        'source': 'data/sample.mp4',  # Will be overridden below
        'frame_skip': 1,
        'process_resolution': [640, 480],
        'output_fps': 30.0
    },
    'detection': {
        'vehicle_model': 'models/yolo11s.onnx',
        'plate_model': 'models/plate_v8n.onnx',
        'vehicle_model_version': 'yolo11',
        'plate_model_version': 'yolov8',
        'confidence': 0.25,
        'iou_threshold': 0.45
    },
    'tracking': {
        'tracker_type': 'bytetrack',
        'confidence': 0.3
    },
    'counting': {
        'use_relative_coordinates': True,
        'line': {
            'start': [0.25, 0.6],
            'end': [0.75, 0.6]
        },
        'raw_coordinates': {
            'start': [320, 360],
            'end': [960, 360]
        }
    },
    'ocr': {
        'languages': ['en'],
        'use_gpu': False
    },
    'hardware': {
        'use_gpu': False,
        'provider': 'auto',
        'precision': 'fp32'
    },
    'storage': {
        'database_path': 'data/traffic_data.db',
        'save_images': False
    },
    'mqtt': {
        'enabled': False,
        'broker': 'localhost',
        'port': 1883,
        'topic_prefix': 'traffic'
    }
}

# Write config to file
with open('config/settings/config.yaml', 'w') as f:
    yaml.dump(config, f, default_flow_style=False)

print("Configuration created successfully")

# Function to update video source
def set_video_source(source_path):
    # Load current config
    with open('config/settings/config.yaml', 'r') as f:
        current_config = yaml.safe_load(f)
    
    # Update video source
    current_config['video']['source'] = source_path
    
    # Write updated config
    with open('config/settings/config.yaml', 'w') as f:
        yaml.dump(current_config, f, default_flow_style=False)
    
    print(f"Video source updated to: {source_path}")

## 5. Select Video Source

Choose which video to process.

In [ ]:
# List available videos
import glob

# Search for videos in different locations
all_videos = []
for ext in ['*.mp4', '*.avi', '*.MOV', '*.mkv']:
    # Look in data directory
    all_videos.extend(glob.glob(os.path.join('data', ext)))
    # For Kaggle, look in input directory if exists
    if IN_KAGGLE and os.path.exists('input'):
        all_videos.extend(glob.glob(os.path.join('input', ext)))
        all_videos.extend(glob.glob(os.path.join('input', '*', ext)))

print("Found videos:")
for i, video in enumerate(all_videos):
    print(f"{i+1}. {video}")

# Allow selection
if all_videos:
    selection = input(f"Enter the number of the video to use (1-{len(all_videos)}): ")
    try:
        idx = int(selection) - 1
        if 0 <= idx < len(all_videos):
            selected_video = all_videos[idx]
            set_video_source(selected_video)
        else:
            print(f"Invalid selection. Using first video: {all_videos[0]}")
            set_video_source(all_videos[0])
    except ValueError:
        print(f"Invalid input. Using first video: {all_videos[0]}")
        set_video_source(all_videos[0])
else:
    print("No videos found. Please upload a video first.")

## 6. Adjust Configuration (Optional)

You can adjust various settings to customize how the system works.

In [ ]:
# Function to update config parameters
def update_config_params(**kwargs):
    # Load current config
    with open('config/settings/config.yaml', 'r') as f:
        config = yaml.safe_load(f)
    
    # Update parameters
    for key, value in kwargs.items():
        if '.' in key:
            # Handle nested parameters like 'video.frame_skip'
            parts = key.split('.')
            current = config
            for part in parts[:-1]:
                current = current.setdefault(part, {})
            current[parts[-1]] = value
        else:
            config[key] = value
    
    # Write updated config
    with open('config/settings/config.yaml', 'w') as f:
        yaml.dump(config, f, default_flow_style=False)
    
    print(f"Configuration updated with: {kwargs}")

# Example: Adjust common parameters
frame_skip = 1  # Process every frame (2 would process every other frame)
detection_confidence = 0.25  # Lower = more detections but more false positives
use_gpu = False  # Set to True if your system has GPU support
count_line_position = 0.6  # Vertical position (0-1) of counting line

update_config_params(
    **{
        'video.frame_skip': frame_skip,
        'detection.confidence': detection_confidence,
        'hardware.use_gpu': use_gpu,
        'counting.line.start': [0.25, count_line_position],
        'counting.line.end': [0.75, count_line_position]
    }
)

## 7. Run Traffic Monitoring System

Now let's run the traffic monitoring system to process the video.

In [ ]:
# Create a modified version of the main application for Jupyter environment
import sys
import os
import time
import cv2
import numpy as np
from IPython.display import clear_output, HTML, display
from base64 import b64encode
from pathlib import Path
import threading
import queue

# Add parent directory to path
sys.path.append(os.path.dirname(os.path.abspath('.')))

# Import services - these imports will work when running inside the traffic_monitoring directory
from services.video_ingestion.service import VideoIngestionService
from services.detection.service import DetectionService
from services.tracking.service import TrackingService
from services.counting.service import CountingService
from services.ocr.service import OCRService
from services.storage.service import StorageService
from config import config

class JupyterTrafficMonitor:
    """Traffic monitoring application adapted for Jupyter environment"""
    
    def __init__(self, video_source=None, record_output=True, output_path=None):
        self.video_source = video_source or config.VIDEO_SOURCE
        self.record_output = record_output
        self.output_path = output_path
        self.running = False
        self.frames = []
        # Initialize services
        print("Initializing services...")
        self.video_service = VideoIngestionService(source=self.video_source)
        self.detection_service = DetectionService()
        self.tracking_service = TrackingService()
        self.counting_service = CountingService()
        self.ocr_service = OCRService()
        self.storage_service = StorageService()
        
        # Video writer for recording output
        self.video_writer = None
        self.video_writer_lock = threading.Lock()
        self.video_queue = queue.Queue(maxsize=30)
        self.video_writer_thread = None
        self.video_writer_running = False
        
        print("Services initialized")
    
    def start(self):
        """Start processing"""
        if self.running:
            print("Already running")
            return
        
        print("Starting traffic monitoring...")
        
        # Start services
        self.storage_service.start()
        self.video_service.start()
        
        # Initialize video writer if recording
        if self.record_output:
            self._init_video_writer()
            if self.video_writer is not None:
                self.video_writer_running = True
                self.video_writer_thread = threading.Thread(target=self._video_writer_thread)
                self.video_writer_thread.daemon = True
                self.video_writer_thread.start()
        
        self.running = True
        return self

In [ ]:
    def _init_video_writer(self):
        """Initialize video writer"""
        # Get first frame to determine dimensions
        frame_data = self.video_service.get_frame()
        if frame_data is None:
            print("Failed to get frame for initializing video writer")
            return
        
        frame = frame_data['frame']
        height, width = frame.shape[:2]
        
        # Create output path
        if self.output_path:
            output_path = self.output_path
            os.makedirs(os.path.dirname(os.path.abspath(output_path)), exist_ok=True)
        else:
            output_dir = Path('data/recordings')
            output_dir.mkdir(parents=True, exist_ok=True)
            timestamp = time.strftime("%Y%m%d_%H%M%S")
            output_path = str(output_dir / f"traffic_monitoring_{timestamp}.mp4")
        
        # Ensure MP4 extension
        if not output_path.lower().endswith('.mp4'):
            output_path = output_path.rsplit('.', 1)[0] + '.mp4'
        
        print(f"Recording output to {output_path}")
        
        # Initialize writer with H264 codec
        try:
            fourcc = cv2.VideoWriter_fourcc(*'H264')
            self.video_writer = cv2.VideoWriter(
                output_path, fourcc, config.OUTPUT_FPS, (width, height))
            
            if not self.video_writer.isOpened():
                # Try MP4V as fallback
                fourcc = cv2.VideoWriter_fourcc(*'MP4V')
                self.video_writer = cv2.VideoWriter(
                    output_path, fourcc, config.OUTPUT_FPS, (width, height))
                
                if not self.video_writer.isOpened():
                    # Try AVI as last resort
                    output_path = output_path.rsplit('.', 1)[0] + '.avi'
                    fourcc = cv2.VideoWriter_fourcc(*'XVID')
                    self.video_writer = cv2.VideoWriter(
                        output_path, fourcc, config.OUTPUT_FPS, (width, height))
                    
                    if not self.video_writer.isOpened():
                        print("Failed to initialize video writer")
                        self.video_writer = None
                        self.record_output = False
        except Exception as e:
            print(f"Error initializing video writer: {e}")
            self.video_writer = None
            self.record_output = False
        
        # Return frame to pipeline
        self.video_service.rewind_one_frame()

In [ ]:
    def _video_writer_thread(self):
        """Thread for writing video frames"""
        while self.video_writer_running:
            try:
                frame = self.video_queue.get(timeout=0.5)
                if frame is None:
                    break
                
                with self.video_writer_lock:
                    if self.video_writer is not None:
                        self.video_writer.write(frame)
                
                self.video_queue.task_done()
            except queue.Empty:
                continue
            except Exception as e:
                print(f"Error in video writer thread: {e}")
    
    def process_frames(self, max_frames=None, display_interval=5):
        """Process video frames and display results at intervals"""
        self.frames = []
        frame_count = 0
        total_frames = self.video_service.get_total_frames()
        
        try:
            while self.running:
                # Get frame
                frame_data = self.video_service.get_frame()
                if frame_data is None:
                    print("Finished processing all frames")
                    break
                
                frame_count += 1
                
                # Process frame
                start_time = time.time()
                
                # Detection
                detection_results = self.detection_service.detect(frame_data)
                
                # Tracking
                tracking_results = self.tracking_service.update(frame_data, detection_results)
                
                # Counting
                counting_results = self.counting_service.update(frame_data, tracking_results)
                
                # OCR
                ocr_results = self.ocr_service.read_plates(frame_data, detection_results, tracking_results)
                
                # Calculate FPS
                processing_time = time.time() - start_time
                fps = 1.0 / processing_time if processing_time > 0 else 0

In [ ]:
                # Prepare visualization
                vis_frame = self._prepare_visualization(
                    frame_data, detection_results, tracking_results, 
                    counting_results, ocr_results, fps
                )
                
                # Store frame for display
                if frame_count % display_interval == 0 or frame_count == 1:
                    self.frames.append(vis_frame)
                    
                    # Display progress
                    if total_frames > 0:
                        progress = (frame_count / total_frames) * 100
                        print(f"Progress: {frame_count}/{total_frames} frames ({progress:.1f}%)")
                    else:
                        print(f"Processed {frame_count} frames")
                    
                    # Show the latest frame
                    self._display_frame(vis_frame)
                
                # Record output if enabled
                if self.record_output and self.video_writer is not None and self.video_writer_running:
                    try:
                        if not self.video_queue.full():
                            self.video_queue.put(vis_frame, block=False)
                    except Exception as e:
                        print(f"Error queuing video frame: {e}")
                
                # Check if max frames reached
                if max_frames and frame_count >= max_frames:
                    print(f"Reached maximum frames: {max_frames}")
                    break
                
        except KeyboardInterrupt:
            print("Processing interrupted")
        except Exception as e:
            print(f"Error processing frames: {e}")
        finally:
            if total_frames > 0:
                print(f"Processed {frame_count}/{total_frames} frames")
            self.stop()
        
        return self.frames

In [ ]:
    def _prepare_visualization(self, frame_data, detection_results, tracking_results, 
                             counting_results, ocr_results, fps):
        """Create visualization frame"""
        # Use original frame for visualization if available to show full resolution
        if 'original_frame' in frame_data:
            frame = frame_data['original_frame'].copy()
            processed_frame = frame_data['frame'].copy()
            
            # Calculate scaling factors
            scale_x = frame.shape[1] / processed_frame.shape[1]
            scale_y = frame.shape[0] / processed_frame.shape[0]
            
            # Function to scale coordinates
            def scale_coords(coords):
                if isinstance(coords, tuple) or isinstance(coords, list):
                    if len(coords) == 2:  # Single point (x,y)
                        return (int(coords[0] * scale_x), int(coords[1] * scale_y))
                    elif len(coords) == 4:  # Box (x1,y1,x2,y2)
                        return (int(coords[0] * scale_x), int(coords[1] * scale_y), 
                                int(coords[2] * scale_x), int(coords[3] * scale_y))
                return coords
            
            # Scale the counting line
            line = counting_results['counting_line']
            line = [scale_coords(line[0]), scale_coords(line[1])]
        else:
            # Fall back to processed frame if original is not available
            frame = frame_data['frame'].copy()
            line = counting_results['counting_line']
            
            def scale_coords(coords):
                return coords  # No scaling needed for processed frame
        
        # Draw counting line
        cv2.line(frame, tuple(line[0]), tuple(line[1]), (0, 0, 255), 2)
        
        # Draw tracks
        for track in tracking_results['tracks']:
            track_id = track['track_id']
            box = scale_coords(track['box'])
            x1, y1, x2, y2 = box
            
            # Color: orange for normal tracks, green for counted tracks
            color = (0, 165, 255)  # Orange for uncounted tracks
            if track_id in self.counting_service.counted_tracks:
                color = (0, 255, 0)  # Green for counted tracks
            
            # Draw bounding box
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
            
            # Draw track ID with better visibility
            text = f"ID:{track_id}"
            text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
            cv2.rectangle(frame, 
                         (int(x1), int(y1) - text_size[1] - 5),
                         (int(x1) + text_size[0] + 5, int(y1)), 
                         (0, 0, 0), -1)
            cv2.putText(frame, text, (int(x1) + 2, int(y1) - 5),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            # Draw path with scaled coordinates
            path = track['path']
            if len(path) >= 2:
                for i in range(1, len(path)):
                    p1 = scale_coords(path[i-1])
                    p2 = scale_coords(path[i])
                    cv2.line(frame, p1, p2, color, 2)

In [ ]:
        # Draw license plates
        for plate in ocr_results['plates']:
            box = scale_coords(plate['box'])
            x1, y1, x2, y2 = box
            text = plate['text']
            
            # Draw plate box
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
            
            # Draw text background
            text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
            cv2.rectangle(frame, 
                         (int(x1), int(y1) - text_size[1] - 10),
                         (int(x1) + text_size[0] + 10, int(y1)), 
                         (0, 0, 0), -1)
            
            # Draw text
            cv2.putText(frame, text, (int(x1) + 5, int(y1) - 5),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        # Add timestamp
        timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(frame_data['timestamp']))
        cv2.putText(frame, timestamp, (20, frame.shape[0] - 20),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        # Draw counts as overlay
        counts = counting_results['counts']
        overlay = frame.copy()
        
        # Draw semi-transparent background
        cv2.rectangle(overlay, (10, 10), (300, 70), (0, 0, 0), -1)
        cv2.addWeighted(overlay, 0.6, frame, 0.4, 0, frame)
        
        # Draw total count text - larger and more prominent
        cv2.putText(frame, f"TOTAL COUNT: {counts['total']}", (20, 50),
                   cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2)
        
        # Draw FPS
        cv2.putText(frame, f"FPS: {fps:.1f}", (frame.shape[1] - 150, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        
        return frame

In [ ]:
    def _display_frame(self, frame):
        """Display a frame in the Jupyter notebook"""
        # Convert to JPEG
        _, jpeg = cv2.imencode('.jpg', frame)
        # Convert to base64
        jpeg_b64 = b64encode(jpeg).decode('utf-8')
        
        # Display
        clear_output(wait=True)
        display(HTML(f'''
            <div style="text-align:center;">
                <img src="data:image/jpeg;base64,{jpeg_b64}" style="max-width:100%;" />
            </div>
        '''))
    
    def stop(self):
        """Stop processing"""
        if not self.running:
            return
        
        self.running = False
        print("Stopping traffic monitoring...")
        
        # Stop video writer
        if self.video_writer_running:
            self.video_writer_running = False
            self.video_queue.put(None)  # Signal to exit
            if self.video_writer_thread:
                self.video_writer_thread.join(timeout=2.0)
        
        if self.video_writer is not None:
            with self.video_writer_lock:
                self.video_writer.release()
                self.video_writer = None
        
        # Stop services
        self.video_service.stop()
        self.storage_service.stop()
        
        print("Traffic monitoring stopped")

In [ ]:
    def create_video_from_frames(self, output_path=None, fps=30):
        """Create a video from processed frames"""
        if not self.frames:
            print("No frames to create video from")
            return
        
        if output_path is None:
            output_dir = Path('data/recordings')
            output_dir.mkdir(parents=True, exist_ok=True)
            timestamp = time.strftime("%Y%m%d_%H%M%S")
            output_path = str(output_dir / f"traffic_monitoring_summary_{timestamp}.mp4")
        
        # Get dimensions from first frame
        height, width = self.frames[0].shape[:2]
        
        # Create writer
        fourcc = cv2.VideoWriter_fourcc(*'MP4V')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        
        # Write frames
        for frame in self.frames:
            out.write(frame)
        
        out.release()
        print(f"Created summary video with {len(self.frames)} frames at {output_path}")
        return output_path

In [ ]:
# Run the traffic monitoring system
monitor = JupyterTrafficMonitor().start()

# Process frames (set max_frames to limit processing, or None for all frames)
# display_interval controls how often to show progress (higher = faster processing, fewer updates)
frames = monitor.process_frames(max_frames=None, display_interval=10)

# Create a summary video with key frames
summary_video = monitor.create_video_from_frames(fps=5)

print("Processing complete!")

## 8. Download Results

If you're using Colab or Kaggle, you can download the processed video and any data files.

In [ ]:
# Download results
if IN_COLAB:
    from google.colab import files
    
    # Function to download a file
    def download_file(file_path):
        if os.path.exists(file_path):
            files.download(file_path)
            print(f"Downloading {file_path}")
        else:
            print(f"File not found: {file_path}")
    
    # Find output videos
    output_videos = glob.glob('data/recordings/*.mp4')
    for video in output_videos:
        print(f"Found output video: {video}")
    
    # Download the latest video
    if output_videos:
        latest_video = max(output_videos, key=os.path.getmtime)
        download_file(latest_video)
    
    # Download the database
    download_file('data/traffic_data.db')
else:
    print("Files saved to:")
    !ls -lh data/recordings/ | grep mp4

## 9. Conclusion

You've successfully run the Traffic Monitoring System in a Jupyter environment! Here's what we accomplished:

1. Set up the necessary environment
2. Downloaded and prepared AI models
3. Configured the system
4. Processed a video to detect, track, count vehicles and read license plates
5. Recorded and saved the results

This notebook makes it easy to use the Traffic Monitoring System on platforms like Google Colab and Kaggle without needing to install anything locally.

### Next Steps

- Try the system with different videos
- Adjust configuration parameters for better performance
- Explore the database for detailed analysis
- Check out the full documentation in the [GitHub repository](https://github.com/yourusername/traffic_monitoring)